## Query to SQlite database

In this notebook, we will convert the csv-files from Main Project file into an Sqlite database. This enables us to quickly and efficiently retrieve the amount of active job ads, per region and occupational group, from the combined database. The first step is to create the relevant tables in the Sqlite database. Secondly, we will insert the relevant data into the tables, and finally join them into one structured table.

In [1]:
import csv
import sqlite3

In [17]:
# Establishing a connection to local SQlite Database
conn = sqlite3.connect("/Users/wilhelmakesson/Desktop/Master Thesis/Data/muni_int/muni.db")
cursor = conn.cursor()

In [6]:
# Creating a new Table in sqlite db with data over digital job ads
Ads_table = """
    CREATE TABLE IF NOT EXISTS past_ads(
    headline TEXT NOT NULL,
    number_of_vacancies INTEGER NOT NULL,
    publication_date TEXT NOT NULL,
    application_deadline TEXT NOT NULL,
    muni_code TEXT NOT NULL);"""
cursor.execute(Ads_table)
conn.commit()

In [11]:
# Creating a new Table in sqlite db with data over municipal/Regional data addhering to the digital job ads
muni_table = """
    CREATE TABLE IF NOT EXISTS municip(
    municipality_code INTEGER NOT NULL,
    municipality_name TEXT NOT NULL,
    municipality_group_name TEXT NOT NULL,
    county_name TEXT NOT NULL);"""
cursor.execute(muni_table)
conn.commit()

In [8]:
# Importing csv files into SQlite digital job ads table
for file in range(2006, 2021):
    with open(f'/Users/wilhelmakesson/Desktop/Master Thesis/Data/muni_int/{file}_municode.csv', 'r') as csv_file:
        dr = csv.DictReader(csv_file, fieldnames=['headline','number_of_vacancies','publication_date','application_deadline', 'muni_code'])
        to_db = [(i['headline'], i['number_of_vacancies'], i['publication_date'], i['application_deadline'], i['muni_code']) for i in dr]
    cursor.executemany("INSERT INTO past_ads (headline, number_of_vacancies, publication_date, application_deadline, muni_code) VALUES (?, ?, ?, ?, ?);", to_db)
    conn.commit()

In [15]:
# Importing csv file with municipal codes into SQlite municipal data table
with open('/Users/wilhelmakesson/Desktop/Master Thesis/Data/muni_int/Kommungruppsindelningen.csv', 'r') as csv_file:
    dr = csv.DictReader(csv_file)
    to_db = [(i['municipality_code'], i['municipality_name'], i['municipality_group_name'], i['county_name']) for i in dr]
cursor.executemany("INSERT INTO municip (municipality_code, municipality_name, municipality_group_name, county_name) VALUES (?, ?, ?, ?);", to_db)
conn.commit()

In [28]:
#Joining the two tables above into one structured table
join_table = """
    CREATE TABLE IF NOT EXISTS joined_table(
    headline TEXT NOT NULL,
    number_of_vacancies INTEGER NOT NULL,
    publication_date TEXT NOT NULL,
    application_deadline TEXT NOT NULL,
    municipality_name TEXT NOT NULL,
    municipality_group_name TEXT NOT NULL,
    county_name TEXT NOT NULL);"""
cursor.execute(join_table)
conn.commit()

joined_table = """
    INSERT INTO joined_table
    SELECT p.headline, p.number_of_vacancies, p.publication_date, p.application_deadline, 
    m.municipality_name, m.municipality_group_name, m.county_name
    FROM past_ads p INNER JOIN municip m ON p.muni_code = m.municipality_code;"""
cursor.execute(joined_table)
conn.commit()
conn.close()

The data has now successfully been transfered to one sqlite table, and this will be used in the Main Project file for further analysis. 